# Multi-layer neural networks, part 2

In this notebook, we continue our exploration of multi-layer networks and basic principles of deep learning.

## 1. Implementing and fitting a "large" multi-layer neural net

We'll continue working with the fuel efficiency dataset, but this time, instead of designing a network that takes a single input, we'll build a network that accepts _all_ of our input variables.  That is, we'll fit it to our full dataset.

In [ ]:
import numpy as np
import pandas as pd

import plotnine as pn

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import torch
from torch import nn
from skorch import NeuralNetRegressor
from skorch.dataset import CVSplit

In [ ]:
df = pd.read_csv('data/auto_mpg.csv')
#df = pd.read_csv('/blue/zoo4926/share/Jupyter_Content/data/auto_mpg.csv')
df = df.sample(frac=1)


**Exercise:** The code below defines a very simple neural network comprising a singlel neuron that accepts a single input.  Modify the network defined below so that it:
1. Accepts 7 input variables.
2. Has two hidden layers; the first with 100 neurons and the second with 50 neurons.
3. Uses the `nn.ReLU()` transfer function after each hidden layer.

In [ ]:
class BigRegressNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(1, 1)  # This layer has 1 neuron that takes 1 input variable.
        )
    
    def forward(self, X):
        y = self.layers(X)
        
        return y

Now, try fitting your new model to the data.

In [ ]:
x = df.drop(columns=['mpg', 'name']).to_numpy(np.float32)
y = df[['mpg']].to_numpy(np.float32)

nn_model = NeuralNetRegressor(
    BigRegressNet,
    train_split=None,
    batch_size=len(df),
    criterion=nn.MSELoss,
    optimizer=torch.optim.SGD,
    max_epochs=100,
    lr=0.01,
    optimizer__momentum=0.9
)

pl = Pipeline([
    ('scaler', StandardScaler()),
    ('nn', nn_model)
])

pl.fit(x, y)

Let's visualize the loss to see how we're doing.

In [ ]:
tr_df = pd.DataFrame(nn_model.history)

pn.ggplot(tr_df, pn.aes(x='epoch', y='train_loss')) + pn.geom_line() + pn.ylim(0,10)

What do you notice?  Have we found the best model?  Do you have any concerns about the model?

## 2. Using validation data during training

We can get better insight into how our model is performing by monitoring a _validation dataset_ throughout the training process.  In this case, we will create the validation dataset by splitting our full dataset in two.  Let's repeat the fitting procedure implemented above, this time with a validation set.  Use the same learning rate and number epochs you were using above.

Now that we have a validation dataset included in our model training procedure, let's visualize what is happening.

**Exercise:** The validation loss shows a clear pattern of initial poor peformance, followed by rapid improvement, followed by a slower increase in loss.  How would you explain the poor performance on the left-hand side of the graph?  How about on the right-hand side of the graph?  _Try to be as precise as you can in your explanation._  (Hint: Think about the bias-variance decomposition and the two sources of prediction/test error.)

**Exercise:** We didn't seem to have this problem with the neural nets we were fitting before.  Why not?  A related question: How many parameters does our new model have?

**Exercise:** The code we wrote above for graphing training and validation loss will be useful later on in this notebook, too.  Wrap it up inside a Python function that takes the training history from a model and returns a plotnine graph of the training and validation loss.  I've provided you with a partial function implementation to help you get started.

In [ ]:
def graph_tr_history(tr_history, ymin=0, ymax=None):
    tr_df = pd.DataFrame(tr_history)

    if ymax is None:
        ymax = tr_df.train_loss.max()
        
    # What goes here?
    
    return g

In [ ]:
graph_tr_history(nn_model.history, 0, 15)

## 3. Regularization for neural networks

So, as we just saw, we clearly have a problem with our model fitting procedure, and we need to do something about it!

Earlier in the semester, we discussed _regularization_ techniques for fitting machine learning models.  Do you recall what "regularization" means?  Why do we need it?

There are many regularization methods available for neural networks.  Given time and other constraints, we will only consider a few of them here.

### a. Early stopping

One common approach to fitting a neural network is to save the parameter values that resulted in the lowest validation loss and take that as our final model.  This approach, and variations on it, is a regularization technique called _early stopping_.  The basic idea is that we _stop training once the validation loss is no longer improving_.

**Exercise:** Throughout this course, we've emphasized the importance of obtaining good estimates of model performance on data not used for training.  With early stopping, can we use statistics on our validation set to estimate model performance?  Do you see any connection between the previous question and how we've sometimes been using cross-validation?

**Exercise:** In general, when information from a validation or test set _leaks_ into our training set, it can bias our estimates of model performance.  This problem can be much more subtle then you might realize.  Look again at our model training code.  Can you spot any (other) problems?

### b. Weight decay

Recall that to implement ridge regression, we used our usual regression loss function, $MSE$, but we also defined a penalty term that multiplied the $\ell_2$ norm of the model parameters by a constant, $\lambda$, and added that quantity to the loss.  We can do the _same thing_ with neural networks, and in this new context, the technique is called _weight decay_.  Let's look at how we can use weight decay with pytorch and skorch.

In [ ]:
graph_tr_history(nn_model.history, 0, 15)

**Exercise:** Experiment with the value of the weight decay hyperparameter.  Which value or values seem to work well?  How could you choose a value for this hyperparameter?

**Exercise:** Once you've picked a good value for the weight decay hyperparameter, fit your model several times and observe the results.  What do you notice?

As I mentioned at the beginning of this section, there are _many_ ways to implement regularization when fitting neural networks.  Another important regularization method you should know about is called _data augmentation_.  The idea behind data augmentation is to add artificial training examples to our training dataset which, if done well, should help the network generalize better.  There are many different ways to implement this.

In general, it is common to use multiple regularization techniques in concert when fitting big deep learning models.  Unfortunately, we don't have time to cover more of these techniques in class, but I hope this discussion at least provides motivation and a launch point for learning more.

## 4. Stochastic gradient descent

Thus far, we've been using our full training dataset to calculate the value of our loss function and update our model parameters via gradient descent.  When training large neural nets, it is common to use a variation of this technique called _stochastic_ gradient descent where we split our training data up into _batches_ (often called "mini-batches") and calculate the loss and parameter updates on each batch separately.

Let's try implementing this for the model above.

**Exercise:** Train your model using a batch size of 32.  (I suggest reducing the number of epochs to ~400 or so to make experimentation easier.)  How does the batch size affect the other training hyperparameters?  You will find this easier to answer if you graph the training and validation losses.

In [ ]:
graph_tr_history(nn_model.history, 0, 15)